In [5]:
import os
# import time
import cv2


In [6]:
# 批量转换图片格式（大小，灰度）
def alter(path,object):
  result = []
  s = os.listdir(path)
  count = 1
  for i in s:
    document = os.path.join(path,i)
    img = cv2.imread(document)
#     img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY )
    img = cv2.resize(img, (64,64))
    listStr = [str(count)]
    fileName = ''.join(listStr)
    cv2.imwrite(object+os.sep+'%s.jpg' % fileName, img)
    count = count + 1
alter('raw_dataset/0','raw_dataset/3')

In [17]:
# 扩充数据集
# -*- coding: utf-8 -*-

 
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import tensorflow as tf
 
def creat_x_database(rootdir,resize_row,resize_col):
    list = os.listdir(rootdir)
    database=np.arange(len(list)*resize_row*resize_col*3).reshape(len(list),resize_row,resize_col,3)
    for i in range(0,len(list)):
        path = os.path.join(rootdir,list[i])    #把目录和文件名合成一个路径
        if os.path.isfile(path):                ##判断路径是否为文件
            image_raw_data = tf.gfile.FastGFile(path,'rb').read()
            with tf.Session() as sess:
                img_data = tf.image.decode_jpeg(image_raw_data)
                resized=tf.image.resize_images(img_data,[resize_row,resize_col],method=0)
                database[i]=resized.eval()
    return database
 
 
'''
用以生成一个batch的图像数据，支持实时数据提升。训练时该函数会无限生成数据，
直到达到规定的epoch次数为止。
'''
datagen = ImageDataGenerator(
    featurewise_center=True,#去中心化
    featurewise_std_normalization=True,#标准化
    rotation_range=2,#旋转范围, 随机旋转(0-180)度
    width_shift_range=0.1,#随机沿着水平或者垂直方向，以图像的长宽小部分百分比为变化范围进行平移;
    height_shift_range=0.1,
    shear_range=0.1,#水平或垂直投影变换
    zoom_range=0.1,#按比例随机缩放图像尺寸
    horizontal_flip=True,#水平翻转图像
    fill_mode='nearest')#填充像素, 出现在旋转或平移之后
 

 
x=creat_x_database('dataset/1_1/',300,300)
datagen.fit(x)
 
i=0
for batch in datagen.flow(
        x,
        batch_size=3,
        save_to_dir='dataset/1_1_new/',
        save_prefix='',
        save_format='jpg'):
    
    i+=1
    if i>30:
        break